In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import pandas as pd 
import numpy as np
import nltk
from nltk.corpus import stopwords
from tqdm._tqdm_notebook import tqdm_notebook
import warnings
from nltk.tokenize import sent_tokenize

nltk.download('punkt')
warnings.filterwarnings('ignore')
tqdm_notebook.pandas()
nltk.download('stopwords')
i

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


/tmp/ipykernel_33/4037459634.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  from tqdm._tqdm_notebook import tqdm_notebook


True

In [7]:
import os
os.listdir('/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail')

['validation.csv', 'train.csv', 'test.csv']

In [14]:

df = pd.read_csv('/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/test.csv')
print(df.shape)
df.head()

(11490, 3)


,id,article,highlights
0,92c514c913c0bdfe25341af9fd72b29db544099b,Ever noticed how plane seats appear to be gett...,Experts question if packed out planes are put...
1,2003841c7dc0e7c5b1a248f9cd536d727f27a45a,A drunk teenage boy had to be rescued by secur...,Drunk teenage boy climbed into lion enclosure ...
2,91b7d2311527f5c2b63a65ca98d21d9c92485149,Dougie Freedman is on the verge of agreeing a ...,Nottingham Forest are close to extending Dougi...
3,caabf9cbdf96eb1410295a673e953d304391bfbb,Liverpool target Neto is also wanted by PSG an...,Fiorentina goalkeeper Neto has been linked wit...
4,3da746a7d9afcaa659088c8366ef6347fe6b53ea,Bruce Jenner will break his silence in a two-h...,"Tell-all interview with the reality TV star, 6..."


### Preprocessing

In [16]:
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are"
}

In [21]:
import re 
def clean_text(text, remove_stopwords = True):
    '''Remove unwanted characters, stopwords, and format the text to create fewer nulls word embeddings'''
    
    # Convert words to lower case
    text = text.lower()
    
    # Replace contractions with their longer forms 
    if True:
        text = text.split()
        new_text = []
        for word in text:
            if word in contractions:
                new_text.append(contractions[word])
            else:
                new_text.append(word)
        text = " ".join(new_text)
    
    # Format words and remove unwanted characters
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)

    text = re.sub(r'\'', ' ', text)
    
    # Optionally, remove stop words
    if remove_stopwords:
        text = text.split()
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
        text = " ".join(text)

    return text

In [22]:
clean_highlights = []
for i in df.highlights:
    clean_highlights.append(clean_text(i, remove_stopwords=False))
print("Summaries are complete.")

clean_article = []
for k in df.article:
    clean_article.append(clean_text(k))
print("Texts are complete.")
     

Summaries are complete.
Texts are complete.


In [27]:
PERCENT_RATIO = 0.2

### TextRank

In [29]:
pip install summa

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for summa: filename=summa-1.2.0-py3-none-any.whl size=54388 sha256=3badb02f7eafab6e17c0f609aa7ec735b73b91a1c09c8278dae723e511f1805a
  Stored in directory: /root/.cache/pip/wheels/4a/ca/c5/4958614cfba88ed6ceb7cb5a849f9f89f9ac49971616bc919f
Successfully built summa
Note: you may need to restart the kernel to use updated packages.


In [33]:
df_need = pd.DataFrame({'article':clean_article,'highlights':clean_highlights})
df_need.head()

,article,highlights
0,ever noticed plane seats appear getting smalle...,experts question if packed out planes are putt...
1,drunk teenage boy rescued security jumping lio...,drunk teenage boy climbed into lion enclosure ...
2,dougie freedman verge agreeing new two-year de...,nottingham forest are close to extending dougi...
3,liverpool target neto also wanted psg clubs sp...,fiorentina goalkeeper neto has been linked wit...
4,bruce jenner break silence two-hour interview ...,"tell-all interview with the reality tv star, 6..."


In [35]:
from summa.summarizer import summarize 
df_need['text_rank'] = df_need['article'].progress_apply(lambda x:summarize(x,ratio = PERCENT_RATIO))

  0%|          | 0/11490 [00:00<?, ?it/s]

### LexRank

In [37]:
pip install lexrank

Note: you may need to restart the kernel to use updated packages.


In [46]:
from lexrank import LexRank
lxr = LexRank(list(df_need['article']))


In [48]:

print("Tokenization")
df_need['article_sentences'] = df_need['article'].progress_apply(lambda x:sent_tokenize(x))
print("LexRank algo")
df_need['lex_rank'] = df_need['article_sentences']\
.progress_apply(lambda x: lxr\
                .get_summary(x,
                             summary_size=round(PERCENT_RATIO*len(x)+1)
                             ,threshold=.1
                            )
               )
def connect(x:list[str])->str:
    return ''.join(x)

df_need['lex_rank'] = df_need['lex_rank'].apply(connect)

Tokenization


  0%|          | 0/11490 [00:00<?, ?it/s]

LexRank algo


  0%|          | 0/11490 [00:00<?, ?it/s]

### Evaluation

In [56]:
pip install rouge-score

Note: you may need to restart the kernel to use updated packages.


In [62]:
import rouge_score as rouge
from rouge_score.rouge_scorer import RougeScorer

rouge_metrics = RougeScorer(['rouge1', 'rougeL','rouge2'], use_stemmer=True)
df_need['text_rank_scoresL'] = df_need[['highlights','text_rank']]\
.apply(lambda x: rouge_metrics.score(x['highlights'],x['text_rank'])['rougeL'].fmeasure,axis = 1)

df_need['lex_rank_scoresL'] = df_need[['highlights','lex_rank']]\
.apply(lambda x: rouge_metrics.score(x['highlights'],x['lex_rank'])['rougeL'].fmeasure,axis = 1)




df_need['text_rank_scores1'] = df_need[['highlights','text_rank']]\
.apply(lambda x: rouge_metrics.score(x['highlights'],x['text_rank'])['rouge1'].fmeasure,axis = 1)

df_need['lex_rank_scores1'] = df_need[['highlights','lex_rank']]\
.apply(lambda x: rouge_metrics.score(x['highlights'],x['lex_rank'])['rouge1'].fmeasure,axis = 1)


df_need['text_rank_scores2'] = df_need[['highlights','text_rank']]\
.apply(lambda x: rouge_metrics.score(x['highlights'],x['text_rank'])['rouge2'].fmeasure,axis = 1)

df_need['lex_rank_scores2'] = df_need[['highlights','lex_rank']]\
.apply(lambda x: rouge_metrics.score(x['highlights'],x['lex_rank'])['rouge2'].fmeasure,axis = 1)

In [63]:

from scipy import stats

def calculate_confidence_interval(data, confidence=0.95):
    mean = np.mean(data)
    n = len(data)
    stderr = stats.sem(data)
    interval = stderr * stats.t.ppf ((1 + confidence) / 2., n - 1)
    return mean - interval, mean ,mean + interval

In [65]:
print("Lex L:",calculate_confidence_interval(df_need['lex_rank_scoresL']))
print("Text L: ",calculate_confidence_interval(df_need['text_rank_scoresL']))


print("Lex 1:",calculate_confidence_interval(df_need['lex_rank_scores1']))
print("Text 1: ",calculate_confidence_interval(df_need['text_rank_scores1']))


print("Lex 2:",calculate_confidence_interval(df_need['lex_rank_scores2']))
print("Text 2: ",calculate_confidence_interval(df_need['text_rank_scores2']))

Lex L: (0.15755122356448806, 0.1587686090100962, 0.15998599445570436)
Text L:  (0.1737728214706593, 0.17517341531789618, 0.17657400916513305)
Lex 1: (0.23647693271625517, 0.23808666028574965, 0.23969638785524414)
Text 1:  (0.24480137654801817, 0.24650273513995805, 0.24820409373189792)
Lex 2: (0.07130023370162704, 0.07219786527425472, 0.0730954968468824)
Text 2:  (0.07640061107910762, 0.0773692052334179, 0.0783377993877282)
